## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import sys
import os

# Add src directory to path
sys.path.append('../src')

# Import preprocessing functions
from preprocess import preprocess_text, preprocess_skills_column

# Import ML libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

print("Libraries imported successfully!")

## 2. Load Dataset

In [ ]:
# Load the dataset
df = pd.read_csv('../dataset/skills_dataset.csv')
print(f"Dataset loaded: {df.shape[0]} records, {df.shape[1]} columns")
df.head()

## 3. Text Preprocessing

In [ ]:
# Show original skills examples
print("Original Skills (sample):")
print(df['skills'].head(10).to_string())

In [ ]:
# Apply preprocessing to skills column
df['skills_clean'] = df['skills'].apply(preprocess_text)
print("Preprocessing completed!")
print("\nCleaned Skills (sample):")
print(df['skills_clean'].head(10).to_string())

In [ ]:
# Compare original vs cleaned
comparison = pd.DataFrame({
    'Original': df['skills'].head(10),
    'Cleaned': df['skills_clean'].head(10)
})
print("Before and After Preprocessing:")
comparison

## 4. TF-IDF Feature Extraction

In [ ]:
# Initialize TF-IDF Vectorizer
tfidf = TfidfVectorizer(
    max_features=100,  # Limit to top 100 features
    ngram_range=(1, 2),  # Use unigrams and bigrams
    min_df=1,  # Minimum document frequency
    max_df=0.8  # Maximum document frequency (ignore very common terms)
)

print("TF-IDF Vectorizer initialized")
print(f"Configuration: max_features=100, ngram_range=(1,2)")

In [ ]:
# Fit and transform the cleaned skills
X = tfidf.fit_transform(df['skills_clean'])

print(f"Feature matrix shape: {X.shape}")
print(f"Number of samples: {X.shape[0]}")
print(f"Number of features: {X.shape[1]}")

In [ ]:
# Get feature names
feature_names = tfidf.get_feature_names_out()
print(f"\nTotal features extracted: {len(feature_names)}")
print(f"\nFirst 20 features:")
print(feature_names[:20])

In [ ]:
# Convert to DataFrame for better visualization
X_df = pd.DataFrame(X.toarray(), columns=feature_names)
print("\nFeature matrix (first 5 rows, first 10 features):")
X_df.iloc[:5, :10]

## 5. Label Encoding

In [ ]:
# Initialize Label Encoder
label_encoder = LabelEncoder()

# Encode target variable
y = label_encoder.fit_transform(df['job_role'])

print(f"Target variable encoded: {len(y)} labels")
print(f"Number of unique classes: {len(label_encoder.classes_)}")
print(f"\nClass labels: {label_encoder.classes_}")

In [ ]:
# Show encoding mapping
encoding_map = pd.DataFrame({
    'Job Role': label_encoder.classes_,
    'Encoded Label': range(len(label_encoder.classes_))
})
print("\nJob Role to Label Mapping:")
encoding_map

## 6. Train-Test Split

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y  # Maintain class distribution
)

print("Data split completed!")
print(f"\nTraining set: {X_train.shape[0]} samples")
print(f"Testing set: {X_test.shape[0]} samples")
print(f"Feature dimensions: {X_train.shape[1]}")

In [ ]:
# Check class distribution in train and test sets
print("Training set class distribution:")
train_dist = pd.Series(y_train).value_counts().sort_index()
print(train_dist)

print("\nTesting set class distribution:")
test_dist = pd.Series(y_test).value_counts().sort_index()
print(test_dist)

## 7. Feature Analysis

In [ ]:
# Analyze top features
feature_importance = np.asarray(X.sum(axis=0)).flatten()
feature_df = pd.DataFrame({
    'Feature': feature_names,
    'Total Score': feature_importance
}).sort_values('Total Score', ascending=False)

print("\nTop 20 Most Important Features:")
feature_df.head(20)

In [ ]:
# Visualize top 15 features
plt.figure(figsize=(12, 6))
top_features = feature_df.head(15)
plt.barh(top_features['Feature'], top_features['Total Score'], color='teal')
plt.xlabel('TF-IDF Score (Sum)', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('Top 15 Most Important Features (TF-IDF)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 8. Summary Statistics

In [ ]:
# Generate summary statistics
summary = {
    'Total Samples': len(df),
    'Total Features': X.shape[1],
    'Number of Classes': len(label_encoder.classes_),
    'Training Samples': X_train.shape[0],
    'Testing Samples': X_test.shape[0],
    'Feature Matrix Sparsity': f"{(1 - X.nnz / (X.shape[0] * X.shape[1])) * 100:.2f}%"
}

print("\n" + "="*50)
print("FEATURE ENGINEERING SUMMARY")
print("="*50)
for key, value in summary.items():
    print(f"{key:.<30} {value}")
print("="*50)

## 9. Save Feature Information

In [ ]:
# Create results directory if it doesn't exist
os.makedirs('../results', exist_ok=True)

# Save feature information to file
with open('../results/feature_info.txt', 'w') as f:
    f.write("="*60 + "\n")
    f.write("FEATURE ENGINEERING SUMMARY - DAY 2\n")
    f.write("Student Skill Gap Analyzer & Career Recommendation System\n")
    f.write("="*60 + "\n\n")
    
    f.write("DATASET INFORMATION:\n")
    f.write("-" * 60 + "\n")
    f.write(f"Total Samples: {len(df)}\n")
    f.write(f"Number of Job Roles: {len(label_encoder.classes_)}\n")
    f.write(f"Job Roles: {', '.join(label_encoder.classes_)}\n\n")
    
    f.write("FEATURE EXTRACTION (TF-IDF):\n")
    f.write("-" * 60 + "\n")
    f.write(f"Total Features Extracted: {X.shape[1]}\n")
    f.write(f"Max Features Limit: 100\n")
    f.write(f"N-gram Range: (1, 2)\n")
    f.write(f"Feature Matrix Shape: {X.shape}\n")
    f.write(f"Sparsity: {(1 - X.nnz / (X.shape[0] * X.shape[1])) * 100:.2f}%\n\n")
    
    f.write("TRAIN-TEST SPLIT:\n")
    f.write("-" * 60 + "\n")
    f.write(f"Training Samples: {X_train.shape[0]}\n")
    f.write(f"Testing Samples: {X_test.shape[0]}\n")
    f.write(f"Test Size: 20%\n")
    f.write(f"Random State: 42\n")
    f.write(f"Stratified: Yes\n\n")
    
    f.write("TOP 20 FEATURES:\n")
    f.write("-" * 60 + "\n")
    for idx, row in feature_df.head(20).iterrows():
        f.write(f"{row['Feature']:.<40} {row['Total Score']:.4f}\n")

print("Feature information saved to: ../results/feature_info.txt")

## 10. Key Observations

**Preprocessing Achievements:**
- Successfully cleaned skill text (lowercase, removed special characters, normalized spacing)
- Converted text data to numerical features using TF-IDF
- Encoded job roles as numerical labels

**Feature Engineering Results:**
- Extracted meaningful features representing skills and technologies
- Created sparse feature matrix suitable for machine learning
- Maintained class distribution through stratified splitting

**Next Steps (Day 3):**
1. Build baseline classification models
2. Evaluate model performance
3. Experiment with different algorithms
4. Fine-tune hyperparameters